### Import libraries

In [1]:
import pandas as pd
import numpy as np
import os
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

from src.lstm_ts import LSTMTimeseries
from src import MPN_Bogging, MPN_LHDrill, MPN_Trucking, MPN_Jumbo
from src.preprocessing import preprocessing


import warnings
warnings.filterwarnings('ignore')

import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

%load_ext autoreload
%autoreload 2

### Config

In [2]:
freq = 'W'
""" The default frequency for the forecast model """

date_column = 'DATE'
""" The date column for the model """

input_chunk_length = 12
""" The length of the input chunks, required for the forecast model """

forecast_length = 12
""" The default forecast length for the model """

targets = {
    "jumbo": "DRIVINGADVANCE",
    "bogging": "PRIMARY_TONNES",
    "lhdrilling": "LONGHOLEMETRES",
    "trucking": "TKM",
    "trucking_surface_ore": "ORE_TONNES_TO_SURFACE",
}
""" The target columns required by the model """

new_dates_prediction = True

num_test_prediction = 5
""" Backtesting Period Count """

site_name = 'Idobamine'
""" Mine site name """

activity_list = ['trucking', 'bogging', 'jumbo', 'lhdrilling']
""" All activities in the MPN model """

start_date='2021-01-01'

### Create required folders

In [3]:
input_folder = 'bdm1_shiftly_processed'
""" Contains datasets used in the project. """

preprocessed_data_folder = 'preprocessed_data'
""" Contains preprocessed datasets. """

models = 'models'
""" Where trained models are saved. """

results = 'results'
""" Stores the output of model predictions and evaluations in html format. """

# Create directories if they don't exist
os.makedirs(preprocessed_data_folder, exist_ok=True)
os.makedirs(models, exist_ok=True)
os.makedirs(results, exist_ok=True)

In [4]:
def train_models(df, target_column, activity, site_name, num_test_prediction):
    total_pred_df = pd.DataFrame()
    models_name = set()
    for i in range(not new_dates_prediction, num_test_prediction+1):
        logging.info(f'{i+1} Backtest period, {20 * "+"} LSTM model for {activity.upper()} in {site_name.upper()} Site {20 * "+"}')

        #### train and test split ###
        train_idx_max = None if i == 0 else -i*forecast_length
        train_df = df[:train_idx_max]
        train_df = train_df[train_df[date_column] <= train_df[date_column].iloc[-1]]

        ### make target smoother, in this case, alpha=0.4 means that more weight is given to more recent observations.
        train_df[target_column] = train_df[target_column].ewm(alpha=0.4).mean()

        # define test date
        test_dates = pd.date_range(start=train_df[date_column].iloc[-1], periods=forecast_length+1, freq=freq)[1:]
        test_dates = pd.DataFrame({date_column:test_dates})
        pred_df = pd.DataFrame({date_column:test_dates[date_column]})

        ### LSTM train and prediction
        feature_importance = False #True if i == num_test_prediction else False
        lstm = LSTMTimeseries()
        _ = lstm.train(train_df.copy(deep=True), target_column=target_column, date_column=date_column,
                        lookback=input_chunk_length, prediction_length=forecast_length, 
                        save_to=f'{models}/lstm_{site_name}_{activity}.pkl', num_epochs=1068, lr=0.00595,
                        num_layers=2, hidden_dim=43, feature_importance=feature_importance)
        lstm_preds = lstm.predict(model_path=f'{models}/lstm_{site_name}_{activity}.pkl')
        pred_df['LSTM'] = lstm_preds['pred']
        pred_df['LSTM_low'] = lstm_preds['pred_low']
        pred_df['LSTM_high'] = lstm_preds['pred_high']
        models_name.add('LSTM')

        # concatenate predictions
        if len(total_pred_df) == 0:
            for col_name in pred_df.keys():
                total_pred_df[col_name] = []
        total_pred_df = pd.concat([pred_df, total_pred_df])

    total_pred_df = total_pred_df.reset_index(drop=True)

    models_name = list(models_name)

    return total_pred_df, models_name

In [5]:
def plot_prediction(total_pred_df, models_name, target_column, activity, site_name, num_test_prediction):
    ### show predictions
    fig = make_subplots(rows=2, cols=1, subplot_titles=(f'{activity.upper()} in {site_name.upper()}', 'Total of 12 Weeks'))
    
    fig.add_trace(go.Scatter(x=df[date_column], y=df[target_column], 
                                name='actual', mode='lines',
                                marker_color=px.colors.qualitative.Alphabet[0]), row=1, col=1)
    fig.add_trace(go.Scatter(x=[df[date_column].iloc[-forecast_length*num_test_prediction],
                                df[date_column].iloc[-forecast_length*num_test_prediction]],
                                y=[df[target_column].max(),df[target_column].min()],
                                line=dict(dash='dash', color='black'),text=["train and test spliter"],
                                textposition="top center", name='spliter', mode='lines+text'), row=1, col=1)
    c = 1
    for col_name in models_name:
        fig.add_trace(go.Scatter(x=total_pred_df[date_column], y=total_pred_df[col_name], 
                                name=f'{col_name}_mean', mode='lines',
                                marker_color=px.colors.qualitative.Alphabet[c]), row=1, col=1)
        fig.add_trace(go.Scatter(x=total_pred_df[date_column], y=total_pred_df[f'{col_name}_low'], 
                                name=f'{col_name}_low', mode='lines', 
                                line=dict(width=0),
                                showlegend=False,
                                marker_color=px.colors.qualitative.Alphabet[c]), row=1, col=1)
        fig.add_trace(go.Scatter(x=total_pred_df[date_column], y=total_pred_df[f'{col_name}_high'], 
                                name=f'{col_name}_uncertrainty', mode='lines',fill='tonexty',line=dict(width=0),
                                marker_color=px.colors.qualitative.Alphabet[c]), row=1, col=1)
        c += 1

    # show bars
    if new_dates_prediction:
        total_pred_df = total_pred_df[:-12]
    total_pred_df['actual'] = df[target_column][-len(total_pred_df):].to_numpy()

    total_pred_df = pd.concat([pd.DataFrame(index=[0]), total_pred_df], ignore_index=True)
    total_pred_df = total_pred_df.set_index('DATE')
    total_pred_df2 = total_pred_df.rolling(12, step=12).sum()[1:]
    total_pred_df2.reset_index(inplace=True)

    fig.add_trace(go.Bar(x=total_pred_df2[date_column] ,y=total_pred_df2['actual'].to_numpy(), 
                        name='Actual, total of 12 weeks',
                        marker_color=px.colors.qualitative.Alphabet[0]), row=2, col=1)
    c = 1
    for col_name in models_name:
        pred = total_pred_df2[col_name].to_numpy()
        mae = 100 * (np.abs(pred - total_pred_df2.actual.values) / total_pred_df2.actual.values)
        mae[mae == np.inf] = 100
        mae = np.mean(mae)
        fig.add_trace(go.Bar(x=total_pred_df2[date_column] ,y=pred, 
                                    name=f'{col_name}, total of 12 weeks, mean_error={round(mae, 1)}%',
                                    marker_color=px.colors.qualitative.Alphabet[c]), row=2, col=1)
        c += 1
    fig.update_layout(width=1500, height=800, legend_traceorder="normal")
    fig.write_html(results + f"/{activity + '_' + site_name}.html")
    fig.show()

In [6]:
for activity in activity_list: 

    input_path = f"{input_folder}/mpn_{site_name}_{activity}.csv"
    save_path = f"{preprocessed_data_folder}/preprocessed_{site_name}_{activity}.csv"
    df = preprocessing(input_path, save_path, activity, site_name, start_date, add_tsfresh_features=False)
    
    # df = pd.read_csv(f'{preprocessed_data_folder}/preprocessed_{site_name}_{activity}.csv')
    df[date_column] = pd.to_datetime(df[date_column])
    target_column = targets[activity]

    total_pred_df, models_name = train_models(df, target_column, activity, site_name, num_test_prediction)
    plot_prediction(total_pred_df, models_name, target_column, activity, site_name, num_test_prediction)

2024-05-07 14:46:01,073 - INFO - 1 Backtest period, ++++++++++++++++++++ LSTM model for TRUCKING in IDOBAMINE Site ++++++++++++++++++++
2024-05-07 14:46:46,775 - INFO - 2 Backtest period, ++++++++++++++++++++ LSTM model for TRUCKING in IDOBAMINE Site ++++++++++++++++++++
2024-05-07 14:47:27,541 - INFO - 3 Backtest period, ++++++++++++++++++++ LSTM model for TRUCKING in IDOBAMINE Site ++++++++++++++++++++
2024-05-07 14:48:04,627 - INFO - 4 Backtest period, ++++++++++++++++++++ LSTM model for TRUCKING in IDOBAMINE Site ++++++++++++++++++++
2024-05-07 14:48:37,266 - INFO - 5 Backtest period, ++++++++++++++++++++ LSTM model for TRUCKING in IDOBAMINE Site ++++++++++++++++++++
2024-05-07 14:49:07,479 - INFO - 6 Backtest period, ++++++++++++++++++++ LSTM model for TRUCKING in IDOBAMINE Site ++++++++++++++++++++


2024-05-07 14:49:36,232 - INFO - 1 Backtest period, ++++++++++++++++++++ LSTM model for BOGGING in IDOBAMINE Site ++++++++++++++++++++
2024-05-07 14:50:18,552 - INFO - 2 Backtest period, ++++++++++++++++++++ LSTM model for BOGGING in IDOBAMINE Site ++++++++++++++++++++
2024-05-07 14:50:55,890 - INFO - 3 Backtest period, ++++++++++++++++++++ LSTM model for BOGGING in IDOBAMINE Site ++++++++++++++++++++
2024-05-07 14:51:29,855 - INFO - 4 Backtest period, ++++++++++++++++++++ LSTM model for BOGGING in IDOBAMINE Site ++++++++++++++++++++
2024-05-07 14:52:04,897 - INFO - 5 Backtest period, ++++++++++++++++++++ LSTM model for BOGGING in IDOBAMINE Site ++++++++++++++++++++
2024-05-07 14:52:34,036 - INFO - 6 Backtest period, ++++++++++++++++++++ LSTM model for BOGGING in IDOBAMINE Site ++++++++++++++++++++


2024-05-07 14:53:01,031 - INFO - 1 Backtest period, ++++++++++++++++++++ LSTM model for JUMBO in IDOBAMINE Site ++++++++++++++++++++
2024-05-07 14:53:41,215 - INFO - 2 Backtest period, ++++++++++++++++++++ LSTM model for JUMBO in IDOBAMINE Site ++++++++++++++++++++
2024-05-07 14:54:18,858 - INFO - 3 Backtest period, ++++++++++++++++++++ LSTM model for JUMBO in IDOBAMINE Site ++++++++++++++++++++
2024-05-07 14:54:52,531 - INFO - 4 Backtest period, ++++++++++++++++++++ LSTM model for JUMBO in IDOBAMINE Site ++++++++++++++++++++
2024-05-07 14:55:23,665 - INFO - 5 Backtest period, ++++++++++++++++++++ LSTM model for JUMBO in IDOBAMINE Site ++++++++++++++++++++
2024-05-07 14:55:54,689 - INFO - 6 Backtest period, ++++++++++++++++++++ LSTM model for JUMBO in IDOBAMINE Site ++++++++++++++++++++


2024-05-07 14:56:21,425 - INFO - 1 Backtest period, ++++++++++++++++++++ LSTM model for LHDRILLING in IDOBAMINE Site ++++++++++++++++++++
2024-05-07 14:57:01,629 - INFO - 2 Backtest period, ++++++++++++++++++++ LSTM model for LHDRILLING in IDOBAMINE Site ++++++++++++++++++++
2024-05-07 14:57:39,967 - INFO - 3 Backtest period, ++++++++++++++++++++ LSTM model for LHDRILLING in IDOBAMINE Site ++++++++++++++++++++
2024-05-07 14:58:12,906 - INFO - 4 Backtest period, ++++++++++++++++++++ LSTM model for LHDRILLING in IDOBAMINE Site ++++++++++++++++++++
2024-05-07 14:58:44,367 - INFO - 5 Backtest period, ++++++++++++++++++++ LSTM model for LHDRILLING in IDOBAMINE Site ++++++++++++++++++++
2024-05-07 14:59:12,180 - INFO - 6 Backtest period, ++++++++++++++++++++ LSTM model for LHDRILLING in IDOBAMINE Site ++++++++++++++++++++


In [7]:
'''def detect_drift(df, target_column, date_column):
    import pandas as pd
    import numpy as np

    # Ensure the date column is in datetime format
    df[date_column] = pd.to_datetime(df[date_column])

    # Drop rows where the target column has NaN values
    df = df.dropna(subset=[target_column])

    # Set the date column as the index
    df.set_index(date_column, inplace=True)

    # Calculate the monthly median of the target column
    monthly_median = df[target_column].resample('M').median()

    # Remove months where the median is NaN after resampling
    monthly_median = monthly_median.dropna()

    # Calculate the gradient of monthly medians
    gradient = monthly_median.diff()

    # Drop NaN values in the gradient to avoid impact in downstream analysis
    gradient = gradient.dropna()

    # Calculate the left rolling average of the gradients over a three-month period
    rolling_average = gradient.rolling(window=3, min_periods=1).mean()

    # Shift the rolling average down by one period to match the current month with the previous month's data
    shifted_rolling_average = rolling_average.shift(1)

    # Drop NaN values in the shifted rolling average to ensure clean data for analysis
    shifted_rolling_average = shifted_rolling_average.dropna()

    # Define a threshold for detecting significant drift
    threshold = 1.00  # Adjust this based on your data sensitivity and scale

    # Determine if there is a significant drift
    drift_detected = (abs(gradient - shifted_rolling_average) > threshold).any()

    difference = abs(gradient - shifted_rolling_average)

    # Calculate the standard deviation of the differences
    std_dev = difference.std()
    
    print(std_dev)

    # Print message if drift is detected
    if drift_detected:
        print(f"Drift detected in {target_column}.")
    else:
        print(f"No drift detected in {target_column}.")

# Example usage:
for activity in activity_list:
    input_path = f"{input_folder}/mpn_{site_name}_{activity}.csv"
    df = pd.read_csv(input_path)
    target_column = targets[activity]
    date_column = 'DATE'  # Assuming 'DATE' is the common date column name
    detect_drift(df, target_column, date_column)
'''

'def detect_drift(df, target_column, date_column):\n    import pandas as pd\n    import numpy as np\n\n    # Ensure the date column is in datetime format\n    df[date_column] = pd.to_datetime(df[date_column])\n\n    # Drop rows where the target column has NaN values\n    df = df.dropna(subset=[target_column])\n\n    # Set the date column as the index\n    df.set_index(date_column, inplace=True)\n\n    # Calculate the monthly median of the target column\n    monthly_median = df[target_column].resample(\'M\').median()\n\n    # Remove months where the median is NaN after resampling\n    monthly_median = monthly_median.dropna()\n\n    # Calculate the gradient of monthly medians\n    gradient = monthly_median.diff()\n\n    # Drop NaN values in the gradient to avoid impact in downstream analysis\n    gradient = gradient.dropna()\n\n    # Calculate the left rolling average of the gradients over a three-month period\n    rolling_average = gradient.rolling(window=3, min_periods=1).mean()\n\n   

In [8]:
'''def detect_drift(df, target_column, date_column):
    import pandas as pd
    import numpy as np

    # Ensure the date column is in datetime format
    df[date_column] = pd.to_datetime(df[date_column])

    # Drop rows where the target column has NaN values
    df = df.dropna(subset=[target_column])

    # Set the date column as the index
    df.set_index(date_column, inplace=True)

    # Calculate the monthly median of the target column
    monthly_median = df[target_column].resample('M').median()

    # Remove months where the median is NaN after resampling
    monthly_median = monthly_median.dropna()

    # Calculate the gradient of monthly medians
    gradient = monthly_median.diff()

    # Drop NaN values in the gradient to avoid impact in downstream analysis
    gradient = gradient.dropna()

    # Calculate the left rolling average of the gradients over a three-month period
    rolling_average = gradient.rolling(window=3, min_periods=1).mean()

    # Shift the rolling average down by one period to match the current month with the previous month's data
    shifted_rolling_average = rolling_average.shift(1)

    # Drop NaN values in the shifted rolling average to ensure clean data for analysis
    shifted_rolling_average = shifted_rolling_average.dropna()

    # Define a threshold for detecting significant drift
    threshold = 2.00  # Adjust this based on your data sensitivity and scale

    # Determine if there is a significant drift
    drift_detected = (abs(gradient - shifted_rolling_average) > threshold).any()

    difference = abs(gradient - shifted_rolling_average)

    # Calculate the standard deviation of the differences
    std_dev = difference.std()
    
    print(std_dev)

        
    # Print message if drift is detected
    if drift_detected:
        print(f"Drift detected in {target_column}. Drift points:")
        drift_points = difference[difference > threshold]
        
        for date, diff in drift_points.items():
            print(f"Date: {date}, Difference: {diff:.4f}")
            
        else:
            print(f"No drift detected in {target_column}.")


# Example usage:
for activity in activity_list:
    input_path = f"{input_folder}/mpn_{site_name}_{activity}.csv"
    df = pd.read_csv(input_path)
    target_column = targets[activity]
    date_column = 'DATE'  # Assuming 'DATE' is the common date column name
    detect_drift(df, target_column, date_column)
'''

'def detect_drift(df, target_column, date_column):\n    import pandas as pd\n    import numpy as np\n\n    # Ensure the date column is in datetime format\n    df[date_column] = pd.to_datetime(df[date_column])\n\n    # Drop rows where the target column has NaN values\n    df = df.dropna(subset=[target_column])\n\n    # Set the date column as the index\n    df.set_index(date_column, inplace=True)\n\n    # Calculate the monthly median of the target column\n    monthly_median = df[target_column].resample(\'M\').median()\n\n    # Remove months where the median is NaN after resampling\n    monthly_median = monthly_median.dropna()\n\n    # Calculate the gradient of monthly medians\n    gradient = monthly_median.diff()\n\n    # Drop NaN values in the gradient to avoid impact in downstream analysis\n    gradient = gradient.dropna()\n\n    # Calculate the left rolling average of the gradients over a three-month period\n    rolling_average = gradient.rolling(window=3, min_periods=1).mean()\n\n   

In [15]:
'''def detect_drift(df, target_column, date_column):
    import pandas as pd
    import numpy as np

    # Ensure the date column is in datetime format
    df[date_column] = pd.to_datetime(df[date_column])

    # Drop rows where the target column has NaN values
    df = df.dropna(subset=[target_column])

    # Set the date column as the index
    df.set_index(date_column, inplace=True)

    # Calculate the monthly median of the target column
    monthly_median = df[target_column].resample('M').median()

    # Remove months where the median is NaN after resampling
    monthly_median = monthly_median.dropna()

    # Calculate the gradient of monthly medians
    gradient = monthly_median.diff()

    # Drop NaN values in the gradient to avoid impact in downstream analysis
    gradient = gradient.dropna()

    # Calculate the left rolling average of the gradients over a three month period
    rolling_average = gradient.rolling(window=3, min_periods=1).mean()

    # Shift the rolling average down by one period to match the current month with the previous month's data
    shifted_rolling_average = rolling_average.shift(1)

    # Drop NaN values in the shifted rolling average to ensure clean data for analysis
    shifted_rolling_average = shifted_rolling_average.dropna()

    # Define a threshold for detecting significant drift
    threshold = 500.00  

    # Determine if there is a significant drift
    drift_detected = (abs(gradient - shifted_rolling_average) > threshold).any()

    difference = abs(gradient - shifted_rolling_average)

    # Calculate the standard deviation of the differences
    std_dev = difference.std()
    
    #print(std_dev)

        
    # Print message if drift is detected
    if drift_detected:
        print(f"Drift detected in {target_column}, in file preprocessed_Idobamine_{activity}.csv,  Drift points:")
        drift_points = difference[difference > threshold]
        
        # If drift points are found, then iterate over them and print
        if not drift_points.empty:
            for date, diff in drift_points.items():
                print(f"Date: {date}, Difference: {diff:.4f}")
                
        else:
            print(f"No significant drift points found above the threshold in {target_column}, in file preprocessed_Idobamine_{activity}.csv")
        
    else:
        print(f"No drift detected in {target_column} in file preprocessed_Idobamine_{activity}.csv.")



# Example usage:
for activity in activity_list:
    input_path = f"{input_folder}/mpn_{site_name}_{activity}.csv"
    df = pd.read_csv(input_path)
    target_column = targets[activity]
    date_column = 'DATE'  
    detect_drift(df, target_column, date_column)
'''

No drift detected in TKM in file preprocessed_Idobamine_trucking.csv.
No drift detected in PRIMARY_TONNES in file preprocessed_Idobamine_bogging.csv.
No drift detected in DRIVINGADVANCE in file preprocessed_Idobamine_jumbo.csv.
No drift detected in LONGHOLEMETRES in file preprocessed_Idobamine_lhdrilling.csv.


In [17]:
def detect_drift(df, target_column, date_column):
    import pandas as pd
    import numpy as np

    # Ensure the date column is in datetime format
    df[date_column] = pd.to_datetime(df[date_column])

    # Drop rows where the target column has NaN values
    df = df.dropna(subset=[target_column])

    # Set the date column as the index
    df.set_index(date_column, inplace=True)

    # Calculate the monthly median of the target column
    monthly_median = df[target_column].resample('M').median()

    # Remove months where the median is NaN after resampling
    monthly_median = monthly_median.dropna()

    # Calculate the gradient of monthly medians
    gradient = monthly_median.diff()

    # Drop NaN values in the gradient to avoid impact in downstream analysis
    gradient = gradient.dropna()

    # Calculate the left rolling average of the gradients over a three-month period
    rolling_average = gradient.rolling(window=3, min_periods=1).mean()

    # Shift the rolling average down by one period to match the current month with the previous month's data
    shifted_rolling_average = rolling_average.shift(1)

    # Drop NaN values in the shifted rolling average to ensure clean data for analysis
    shifted_rolling_average = shifted_rolling_average.dropna()

    # Calculate the absolute difference between the gradient and the shifted rolling average
    difference = abs(gradient - shifted_rolling_average)

    # Standardize the differences using z-score
    difference_z_score = (difference - difference.mean()) / difference.std()

    # Define a threshold for detecting significant drift in z-score terms
    z_threshold = 3.00  # This can be adjusted always i guess **

    # Determine if there is a significant drift
    drift_detected = (difference_z_score > z_threshold).any()

    # Print message if drift is detected
    if drift_detected:
        print(f"Drift detected in {target_column}, in file preprocessed_Idobamine_{activity}.csv. Drift points (Date, Difference, Z-Score):")
        drift_points = difference[difference_z_score > z_threshold]
        
        # If drift points are found, then iterate over them and print
        if not drift_points.empty:
            for date, diff in drift_points.items():
                z_score = difference_z_score[date]
                print(f"Date: {date}, Difference: {diff:.4f}, Z-Score: {z_score:.4f}")
                
        else:
            print(f"No significant drift points found above the threshold in {target_column}, in file preprocessed_Idobamine_{activity}.csv")
        
    else:
        print(f"No drift detected in {target_column} in file preprocessed_Idobamine_{activity}.csv.")

# Example usage:
for activity in activity_list:
    input_path = f"{input_folder}/mpn_{site_name}_{activity}.csv"
    df = pd.read_csv(input_path)
    target_column = targets[activity]
    date_column = 'DATE'  
    detect_drift(df, target_column, date_column)


Drift detected in TKM, in file preprocessed_Idobamine_trucking.csv. Drift points (Date, Difference, Z-Score):
Date: 2019-08-31 00:00:00, Difference: 346.3282, Z-Score: 2.1312
Date: 2019-09-30 00:00:00, Difference: 368.9451, Z-Score: 2.3492
Date: 2022-02-28 00:00:00, Difference: 348.4196, Z-Score: 2.1513
Date: 2023-04-30 00:00:00, Difference: 363.9636, Z-Score: 2.3012
Date: 2023-05-31 00:00:00, Difference: 365.6851, Z-Score: 2.3178
Drift detected in PRIMARY_TONNES, in file preprocessed_Idobamine_bogging.csv. Drift points (Date, Difference, Z-Score):
Date: 2019-08-31 00:00:00, Difference: 478.4673, Z-Score: 3.7422
Date: 2019-11-30 00:00:00, Difference: 483.6698, Z-Score: 3.7928
Date: 2019-12-31 00:00:00, Difference: 344.7201, Z-Score: 2.4415
Date: 2020-01-31 00:00:00, Difference: 324.2096, Z-Score: 2.2420
Drift detected in DRIVINGADVANCE, in file preprocessed_Idobamine_jumbo.csv. Drift points (Date, Difference, Z-Score):
Date: 2020-02-29 00:00:00, Difference: 0.3120, Z-Score: 2.8144
Date